**1) Cargar el archivo en formato texto.**

In [0]:
nasaText = spark.read.text("dbfs:/FileStore/shared_uploads/maria.puche@bosonit.com/access.log")
nasaText.show(5, False)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


**2) División de campos en diferentes columnas con expresiones regulares.**

def regexp_extract(e: Column, exp: String, groupIdx: Int)

In [0]:
from pyspark.sql.functions import regexp_extract

nasa_df = nasaText.select(regexp_extract('value', r'(\S+)', 1).alias('host'),
                         regexp_extract('value', r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2})', 1).alias('timestamp'),
                         regexp_extract('value', r'\"(\S+)\s(\S+)\s*(\S*)\"', 1).alias('method'),
                         regexp_extract('value', r'\"(\S+)\s(\S+)\s*(\S*)\"', 2).alias('endpoint'),
                         regexp_extract('value', r'\"(\S+)\s(\S+)\s*(\S*)\"', 3).alias('protocol'),
                         regexp_extract('value', r'\s(\d{3})\s', 1).cast('integer').alias('status'),
                         regexp_extract('value', r'\s(\S+)$', 1).cast('integer').alias('content_size'))
nasa_df.printSchema()
nasa_df.show(5, False)

root
 |-- host: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- endpoint: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- content_size: integer (nullable = true)

+-----------------+--------------------+------+-----------------------------------------------+--------+------+------------+
|host             |timestamp           |method|endpoint                                       |protocol|status|content_size|
+-----------------+--------------------+------+-----------------------------------------------+--------+------+------------+
|in24.inetnebr.com|01/Aug/1995:00:00:01|GET   |/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|HTTP/1.0|200   |1839        |
|uplherc.upl.com  |01/Aug/1995:00:00:07|GET   |/                                              |HTTP/1.0|304   |0           |
|uplherc.upl.com  |01/Aug/1995:00:00:08|GET   |/images/ksclogo-medium.gif                 

**3) Sustituir los campos vacións por el valor "Sin especificar" y poner timestamp en formato dd/MMM/yyyy:HH:mm:ss.**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

nasa_clean_df = nasa_df.withColumn("host", when(col("host") == "", "Sin especificar").otherwise(col("host")))\
                        .withColumn("timestamp", to_timestamp(col("timestamp"), "dd/MMM/yyyy:HH:mm:ss"))\
                        .withColumn("method", when(col("method") == "", "Sin especificar").otherwise(col("method")))\
                        .withColumn("endpoint", when(col("endpoint") == "", "Sin especificar").otherwise(col("endpoint")))\
                        .withColumn("protocol", when(col("protocol") == "", "Sin especificar").otherwise(col("protocol")))

In [0]:
nasa_clean_df.printSchema()
nasa_clean_df.show(5, False)

root
 |-- host: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- endpoint: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- content_size: integer (nullable = true)

+-----------------+-------------------+------+-----------------------------------------------+--------+------+------------+
|host             |timestamp          |method|endpoint                                       |protocol|status|content_size|
+-----------------+-------------------+------+-----------------------------------------------+--------+------+------------+
|in24.inetnebr.com|1995-08-01 00:00:01|GET   |/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|HTTP/1.0|200   |1839        |
|uplherc.upl.com  |1995-08-01 00:00:07|GET   |/                                              |HTTP/1.0|304   |0           |
|uplherc.upl.com  |1995-08-01 00:00:08|GET   |/images/ksclogo-medium.gif                    

**4) Guardar el nuevo Dataframe en formato parquet con el nombre nasa_parquet.**

In [0]:
nasa_clean_df.write.format("parquet").mode("overwrite").save("/tmp/nasa_parquet/")


**5) Leer el dataset nasa_parquet.**

In [0]:
nasaParquet = spark.read.format("parquet").load("/tmp/nasa_parquet/")
nasaParquet.show(5,False)

+------------------+-------------------+------+----------------------------+--------+------+------------+
|host              |timestamp          |method|endpoint                    |protocol|status|content_size|
+------------------+-------------------+------+----------------------------+--------+------+------------+
|van01028.direct.ca|1995-08-25 21:46:44|GET   |/images/USA-logosmall.gif   |HTTP/1.0|200   |234         |
|van01028.direct.ca|1995-08-25 21:46:45|GET   |/images/WORLD-logosmall.gif |HTTP/1.0|200   |669         |
|205.233.69.3      |1995-08-25 21:46:46|GET   |/ksc.html                   |HTTP/1.0|200   |7089        |
|205.233.69.3      |1995-08-25 21:46:48|GET   |/images/NASA-logosmall.gif  |HTTP/1.0|200   |786         |
|205.233.69.3      |1995-08-25 21:46:48|GET   |/images/MOSAIC-logosmall.gif|HTTP/1.0|200   |363         |
+------------------+-------------------+------+----------------------------+--------+------+------------+
only showing top 5 rows



Una vez que se ha leido el dataset estructurado y depurado, se realizan algunas consultas sobre él.

#### **CONSULTAS**

**6.1) ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.**

In [0]:
nasaParquet.select("protocol").distinct().show()

+---------------+
|       protocol|
+---------------+
|Sin especificar|
|      HTTP/V1.0|
|       HTTP/1.0|
|              a|
+---------------+



**6.2) ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.**

In [0]:
nasaParquet.select("status").groupBy("status") .agg(count("status").alias("total")).orderBy(desc("total")).show()

+------+-------+
|status|  total|
+------+-------+
|   200|1398988|
|   304| 134146|
|   302|  26497|
|   404|  10056|
|   403|    171|
|   501|     27|
|   400|     10|
|   500|      3|
+------+-------+



**6.3) ¿Cuáles son los métodos de petición más utilizados?**

In [0]:
nasaParquet.select("method").groupBy("method").agg(count("method").alias("total")).orderBy(desc("total")).show()

+---------------+-------+
|         method|  total|
+---------------+-------+
|            GET|1564929|
|           HEAD|   3965|
|Sin especificar|    891|
|           POST|    111|
|�|t�9ð'À|u|      2|
+---------------+-------+



**6.4) ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?**

In [0]:
nasaParquet.select("endpoint","content_size").orderBy(desc("content_size")).show(1, False)

+-----------------------------------------------+------------+
|endpoint                                       |content_size|
+-----------------------------------------------+------------+
|/statistics/1995/Jul/Jul95_reverse_domains.html|3421948     |
+-----------------------------------------------+------------+
only showing top 1 row



**6.5) ¿Qué recurso es el que más tráfico recibe?**

In [0]:
nasaParquet.select(col("endpoint")).groupBy("endpoint")\
                                .agg(count("endpoint").alias("total"))\
                                .orderBy(desc("total"))\
                                .show(1,False)

+--------------------------+-----+
|endpoint                  |total|
+--------------------------+-----+
|/images/NASA-logosmall.gif|97384|
+--------------------------+-----+
only showing top 1 row



**6.6) ¿Qué días le web recibió más tráfico?**

In [0]:
nasaParquet.select(col("timestamp")).groupBy(col("timestamp").cast("date").alias("Timestamp"))\
                                    .agg(count("*").alias("Trafico"))\
                                    .orderBy(desc(col("Trafico")))\
                                    .show(5)

+----------+-------+
| Timestamp|Trafico|
+----------+-------+
|1995-08-31|  90125|
|1995-08-30|  80641|
|1995-08-29|  67988|
|1995-08-10|  61248|
|1995-08-11|  61246|
+----------+-------+
only showing top 5 rows



**6.7) ¿A qué horas se produce el mayor número de tráfico en la web?**

In [0]:
nasaParquet.select("timestamp").groupBy(dayofmonth("timestamp").alias("Hours"))\
                                    .agg(count("*").alias("Trafico"))\
                                    .orderBy(desc("Trafico"))\
                                    .show(24)

+-----+-------+
|Hours|Trafico|
+-----+-------+
|   31|  90125|
|   30|  80641|
|   29|  67988|
|   10|  61248|
|   11|  61246|
|    9|  60458|
|    8|  60157|
|   14|  59878|
|    4|  59557|
|   17|  58988|
|   15|  58847|
|   23|  58097|
|   22|  57762|
|    7|  57362|
|   25|  57321|
|   16|  56653|
|   18|  56246|
|   21|  55540|
|   28|  55496|
|   24|  52552|
|    3|  41388|
|   12|  38071|
|   13|  36480|
|    1|  33996|
+-----+-------+
only showing top 24 rows



**6.8) ¿Cuáles son los hosts más frecuentes?**

In [0]:
nasaParquet.select("host").groupBy("host")\
                                    .agg(count("*").alias("Visitas"))\
                                    .orderBy(desc("Visitas"))\
                                    .show(10)

+--------------------+-------+
|                host|Visitas|
+--------------------+-------+
|  edams.ksc.nasa.gov|   6530|
|piweba4y.prodigy.com|   4846|
|        163.206.89.4|   4791|
|piweba5y.prodigy.com|   4607|
|piweba3y.prodigy.com|   4416|
|www-d1.proxy.aol.com|   3889|
|www-b2.proxy.aol.com|   3534|
|www-b3.proxy.aol.com|   3463|
|www-c5.proxy.aol.com|   3423|
|www-b5.proxy.aol.com|   3411|
+--------------------+-------+
only showing top 10 rows



**6.9) ¿Cuál es el número de errores 404 que ha habido cada día?**

In [0]:
nasaParquet.select(col("timestamp"), col("status")).where(col("status") == 404)\
                                    .groupBy(dayofmonth(col("timestamp")).alias("Day"))\
                                    .agg(count("*").alias("Errors"))\
                                    .orderBy(desc(col("Errors")))\
                                    .show(24)

+---+------+
|Day|Errors|
+---+------+
| 30|   571|
|  7|   537|
| 31|   526|
| 29|   420|
| 24|   420|
| 25|   415|
| 28|   410|
|  8|   391|
|  6|   373|
| 27|   370|
| 26|   366|
|  4|   346|
| 23|   345|
| 15|   327|
| 10|   315|
| 20|   312|
| 21|   305|
|  3|   304|
| 22|   288|
| 14|   287|
|  9|   279|
| 17|   271|
| 11|   263|
| 16|   259|
+---+------+
only showing top 24 rows

